Using the CNN weights that were provided by the paper authors. These create a .npy file called `forward_feats.npy`, which should be placed in `setup_existing_model`.

Run the script in `setup_existing_model` first

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
feats = np.load('setup_existing_model/forward_feats.npy')

In [3]:
ims = os.listdir('process_data/data/ims_malawi_2016/')

In [4]:
# since we passed our images to the model in this order, we know index 0
# of this dataframe corresponds to index 0 of feats
df_im_raw = pd.DataFrame.from_dict({'images': ims}); df_im_raw.head()

,images
0,-14.408333_34.099999.png
1,-15.741666_34.491665999999995.png
2,-14.075_35.174999.png
3,-14.233332999999998_34.408332.png
4,-13.766666_33.116665999999995.png


In [5]:
df_im_raw.shape

(23464, 1)

In [6]:
# this index corresponds to the row in "forward_feats.npy"
df_im_raw['feat_index'] = np.arange(len(df_im_raw))

In [7]:
im_to_cons = pd.read_csv('process_data/mw_full_guide.csv')

In [8]:
# notice we have duplicate images that we only pass once to the model to reduce runtime load
# we need to add a column that tells us what index of feats to look at for that image
im_to_cons.head()

,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,clust_num,images_renamed
0,-17.125000,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.125_35.174999.png,0,-17.125_35.174999_0.png
1,-17.133333,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.133333_35.174999.png,0,-17.133333_35.174999_0.png
2,-17.066666,35.191666,-17.09515,35.217213,0.0,2.039307,1,-17.066666_35.191666.png,0,-17.066666_35.191666_0.png
3,-17.050000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.05_35.199999.png,0,-17.05_35.199999_0.png
4,-17.100000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.1_35.199999.png,0,-17.1_35.199999_0.png


In [9]:
im_to_cons.shape

(36546, 10)

In [10]:
# now we can merge
im_to_cons = pd.merge(left=im_to_cons, right=df_im_raw, on='images')

In [11]:
im_to_cons.shape # we shouldn't lose any rows

(36546, 11)

In [12]:
# we now have a dataframe that tells us what index to look at in forward_feats.npy for each image
im_to_cons.head()

,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,clust_num,images_renamed,feat_index
0,-17.125000,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.125_35.174999.png,0,-17.125_35.174999_0.png,9241
1,-17.133333,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.133333_35.174999.png,0,-17.133333_35.174999_0.png,1462
2,-17.066666,35.191666,-17.09515,35.217213,0.0,2.039307,1,-17.066666_35.191666.png,0,-17.066666_35.191666_0.png,7157
3,-17.050000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.05_35.199999.png,0,-17.05_35.199999_0.png,12701
4,-17.100000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.1_35.199999.png,0,-17.1_35.199999_0.png,15357


In [13]:
group = im_to_cons.groupby(['clust_lat', 'clust_lon'])

In [14]:
num_clusts = len(group); num_clusts

778

In [15]:
x = np.zeros((num_clusts, 4096))
y = []

In [16]:
# this goes through each cluster group and finds all images that are in the cluster
# it aggregates the features for those images across the cluster
for i, g in enumerate(group):
    lat, long = g[0]
    im_sub = im_to_cons[(im_to_cons['clust_lat'] == lat) & (im_to_cons['clust_lon'] == long)].reset_index(drop=True)
    agg_feats = np.zeros((len(im_sub), 4096))
    for j, d in im_sub.iterrows():
        agg_feats[j,:] = feats[d.feat_index]
    agg_feats = agg_feats.mean(axis=0) # averages the features across all images in the cluster
    
    x[i,:] = agg_feats
    y.append(g[1]['consumption'].values[0])

In [17]:
y = np.array(y)
y_log = np.log(y) # try predicting consumption and log consumption

In [18]:
x.shape

(778, 4096)

In [19]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

import numpy as np
import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns


def predict_consumption(
    X, y, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    y_hat, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return X, y, y_hat, r2


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
    return y_hat, r2s.mean()


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_hat[test_idx] = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1

In [20]:
_, _, _, r2 = predict_consumption(x, y_log)
r2

best alpha 10.0
best alpha 10.0
best alpha 27.825594022071243
best alpha 10.0
best alpha 10.0


0.1699185557179395

In [21]:
_, _, _, r2 = predict_consumption(x, y)
r2

best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 10.0


0.10141235730129028

<strong> Important Note </strong> <br>
If you read the original paper, you'll see that the paper's consumption prediction has r^2 of 0.33 or higher. Why is ours so much lower using their weights? Well, here we are using (free) 2019 images to predict their 2016 consumption values. Ideally, we should download 2016 images, and if we want to follow the paper exactly, we download 2013 images (for Malawi at least). However, this doesn't explain why if I train the model from scratch, it matches the paper's performance.